In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Gastroesophageal_reflux_disease_(GERD)/GSE68698'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular characterization of systemic sclerosis esophageal pathology identifies inflammatory and proliferative signatures"
!Series_summary	"Esophageal involvement in patients with systemic sclerosis (SSc) is common, but tissue-specific pathological mechanisms are poorly understood. Fibrosis in the esophagus is thought to disrupt smooth muscle function and lead to esophageal dilatation, but autopsy studies demonstrate esophageal smooth muscle atrophy and the absence of fibrosis in the majority of SSc cases. Molecular characterization of SSc esophageal pathology is lacking. Herein, we perform a detailed characterization of SSc esophageal histopathology and molecular signatures at the level of gene expression. Esophageal biopsies were prospectively obtained during esophagogastroduodenoscopy in 16 consecutive SSc patients and 7 subjects without SSc. Upper and lower esophageal biopsies were evaluated for histopathology and gene expression. Individual 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True

# Determine availability of variables
# Sample Characteristics Dictionary
sample_dict = {
    0: ['case/control: control', 'case/control: case'],
    1: ['tissue: esophageal biopsy'],
    2: ['biopsy site: lower', 'biopsy site: upper'],
    3: ['batch: 1', 'batch: 2', 'batch: 3'],
    4: ['systemic sclerosis subtype: NA', 'systemic sclerosis subtype: dcSSc', 'systemic sclerosis subtype: lcSSc'],
    5: ['patient/control id: control 01', 'patient/control id: control 02', 'patient/control id: control 03', 'patient/control id: control 04', 'patient/control id: control 05', 'patient/control id: control 06', 'patient/control id: control 07', 'patient/control id: patient 01', 'patient/control id: patient 02', 'patient/control id: patient 03', 'patient/control id: patient 04', 'patient/control id: patient 05', 'patient/control id: patient 06', 'patient/control id: patient 08', 'patient/control id: patient 09', 'patient/control id: patient 11', 'patient/control id: patient 12', 'patient/control id: patient 13', 'patient/control id: patient 14', 'patient/control id: patient 15', 'patient/control id: patient 17', 'patient/control id: patient 18', 'patient/control id: patient 19']
}

# Check for GERD - assuming from background it's related to the key 0 (case/control)
trait_row = 0

# There is no direct mention of age and gender; assuming data is not available
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    value = value.split(':')[-1].strip()  # Extract the value after colon and remove whitespace
    if value.lower() == 'control':
        return 0  # No GERD
    elif value.lower() == 'case':
        return 1  # GERD
    else:
        return None  # Unknown

def convert_age(value):
    # As age_row is not available, there is no need to define this function
    return None

def convert_gender(value):
    # As gender_row is not available, there is no need to define this function
    return None

# Save metadata
save_cohort_info('GSE68698', './preprocessed/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Gastroesophageal_reflux_disease_(GERD)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Gastroesophageal_reflux_disease_(GERD)/trait_data/GSE68698.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1679252': [0], 'GSM1679253': [0], 'GSM1679254': [0], 'GSM1679255': [0], 'GSM1679256': [0], 'GSM1679257': [0], 'GSM1679258': [0], 'GSM1679259': [0], 'GSM1679260': [0], 'GSM1679261': [0], 'GSM1679262': [0], 'GSM1679263': [0], 'GSM1679264': [0], 'GSM1679265': [1], 'GSM1679266': [1], 'GSM1679267': [1], 'GSM1679268': [1], 'GSM1679269': [1], 'GSM1679270': [1], 'GSM1679271': [1], 'GSM1679272': [1], 'GSM1679273': [1], 'GSM1679274': [1], 'GSM1679275': [1], 'GSM1679276': [1], 'GSM1679277': [1], 'GSM1679278': [1], 'GSM1679279': [1], 'GSM1679280': [1], 'GSM1679281': [1], 'GSM1679282': [1], 'GSM1679283': [1], 'GSM1679284': [1], 'GSM1679285': [1], 'GSM1679286': [1], 'GSM1679287': [1], 'GSM1679288': [1], 'GSM1679289': [1], 'GSM1679290': [1], 'GSM1679291': [1], 'GSM1679292': [1], 'GSM1679293': [1], 'GSM1679294': [1], 'GSM1679295': [1], 'GSM1679296': [1], 'GSM1679297': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100056', 'A_23_P100074', 'A_23_P100092',
       'A_23_P100103', 'A_23_P100111', 'A_23_P100127', 'A_23_P100133',
       'A_23_P100141', 'A_23_P100156', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100341'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the mapping dataframe between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to convert the gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Optionally, preview the gene_data dataframe (can be commented out in production code)
# print("Mapped Gene Data preview:")
# print(preview_df(gene_data))


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Gastroesophageal_reflux_disease_(GERD)/gene_data/GSE68698.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Gastroesophageal_reflux_disease_(GERD)')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE68698', './preprocessed/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Gastroesophageal_reflux_disease_(GERD)/GSE68698.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Gastroesophageal_reflux_disease_(GERD)', the least common label is '0.0' with 13 occurrences. This represents 28.26% of the dataset.
The distribution of the feature 'Gastroesophageal_reflux_disease_(GERD)' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json
